In [75]:
import time, sys, platform
import pyautogui

pyautogui.FAILSAFE = True

def mac_hscroll_pixels(delta_x_px: int, bursts: int = 4, pause: float = 0.02):
    """
    Send horizontal scroll in pixel units on macOS using Quartz.
    Negative delta = scroll/pan left. Positive = right.
    """
    from Quartz import (
        CGEventCreateScrollWheelEvent, CGEventPost,
        kCGHIDEventTap, kCGScrollEventUnitPixel
    )
    step = int(delta_x_px / bursts)
    # CGEventCreateScrollWheelEvent(..., units, wheelCount=2, vertical, horizontal)
    for _ in range(bursts):
        ev = CGEventCreateScrollWheelEvent(
            None, kCGScrollEventUnitPixel, 2, 0, step
        )
        CGEventPost(kCGHIDEventTap, ev)
        time.sleep(pause)

def scroll_left_consistent(start=(500, 700), pixels=300):
    """
    Focus the timeline, pan left by a fixed amount, then return mouse to start.
    Works best on macOS (Quartz). Falls back to a drag method otherwise.
    """
    # 1) Move to known point & click to ensure timeline focus
    pyautogui.moveTo(*start)
    pyautogui.click()
    time.sleep(0.05)

    if platform.system() == "Darwin":
        # 2) Pixel-precise horizontal scroll (negative = left)
        mac_hscroll_pixels(delta_x_px=-abs(pixels), bursts=6, pause=0.02)
    else:
        # 2b) Fallback for non-macOS: drag a scrollbar/thumb if available
        # Adjust Y to your horizontal scrollbar/thumb position
        thumb_x, thumb_y = start[0] + 400, start[1] + 320  # example offset
        pyautogui.moveTo(thumb_x, thumb_y)
        time.sleep(0.05)
        pyautogui.mouseDown(button='left')
        pyautogui.moveRel(-abs(pixels), 0, duration=0.25)
        pyautogui.mouseUp(button='left')

    # 3) Put the mouse back where we started
    pyautogui.moveTo(*start)

# --- Run it:
time.sleep(2)
for _ in range(1):
    time.sleep(0.0)  # give yourself a second to focus Filmora
    scroll_left_consistent(start=(500, 700), pixels=114 * 1000000)


In [76]:
import time, platform, pyautogui

pyautogui.FAILSAFE = True

def mac_hscroll_pixels(delta_x_px: int, bursts: int = 1, pause: float = 0.0):
    """
    Send horizontal scroll in pixel units on macOS using Quartz.
    """
    from Quartz import (
        CGEventCreateScrollWheelEvent, CGEventPost,
        kCGHIDEventTap, kCGScrollEventUnitPixel
    )
    ev = CGEventCreateScrollWheelEvent(
        None, kCGScrollEventUnitPixel, 2, 0, int(delta_x_px)
    )
    CGEventPost(kCGHIDEventTap, ev)
    if pause:
        time.sleep(pause)

def scroll_left_incremental(start=(500, 700), pixels=114, steps=5, pause=0.01):
    """
    Focus the timeline, then scroll left in fixed increments.
    Each step = `pixels`. Run fast with tiny pauses.
    """
    # 1) Focus the timeline
    pyautogui.moveTo(*start)
    pyautogui.click()
    time.sleep(0.05)

    if platform.system() == "Darwin":
        for _ in range(steps):
            mac_hscroll_pixels(-abs(pixels))
            time.sleep(pause)
    else:
        # fallback drag for non-macOS
        thumb_x, thumb_y = start[0] + 400, start[1] + 320
        pyautogui.moveTo(thumb_x, thumb_y)
        pyautogui.mouseDown(button='left')
        for _ in range(steps):
            pyautogui.moveRel(-abs(pixels), 0, duration=0.05)
            time.sleep(pause)
        pyautogui.mouseUp(button='left')

    # 3) Return to starting position
    pyautogui.moveTo(*start)


In [91]:
time.sleep(2)  # let yourself click the window
scroll_left_incremental(start=(500, 700), pixels=114, steps=12 * 5 * 2, pause=0.0005)
